# Objetivo: Previsão de pacientes na UTI (COVID-19)

Utilizarei a base de dados do Hospital Sírio Libanês. A base de dados esta disponível no site do [Kaggle](https://www.kaggle.com/) na pagina do grupo do Sírio Libanês [COVID-19 - Clinical Data to assess diagnosis](https://www.kaggle.com/S%C3%ADrio-Libanes/covid19).

Essa base de dados contém informações, não sensíveis, que diz respeito a quantidade de pacientes que foram ou não internados por covid-19 na clínica do hospital durante a pandemia de corona virus. As informações são ricas com respeito ao quadro clínico e a pergunta que vamos tentar responder é: 

> **Dado um novo paciente conseguiremos prever a chance dele ser encaminhado para a UTI?**


In [1]:
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt

# Importação dos dados e pre processamento

In [19]:
sirio_libanes = pd.read_excel('https://github.com/ConradBitt/BootCamp_DataScience/blob/master/ML%20em%20Saude/dados/Kaggle_Sirio_Libanes_ICU_Prediction.xlsx?raw=true')
sirio_libanes.head()

,PATIENT_VISIT_IDENTIFIER,AGE_ABOVE65,AGE_PERCENTIL,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,...,TEMPERATURE_DIFF,OXYGEN_SATURATION_DIFF,BLOODPRESSURE_DIASTOLIC_DIFF_REL,BLOODPRESSURE_SISTOLIC_DIFF_REL,HEART_RATE_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL,OXYGEN_SATURATION_DIFF_REL,WINDOW,ICU
0,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,0
1,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,2-4,0
2,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4-6,0
3,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,...,-1.000000,-1.000000,NaN,NaN,NaN,NaN,-1.000000,-1.000000,6-12,0
4,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,...,-0.238095,-0.818182,-0.389967,0.407558,-0.230462,0.096774,-0.242282,-0.814433,ABOVE_12,1


In [20]:
def preenche_tabela(dados):
    features_categoricas = dados.iloc[:, :13]
    saida = dados.iloc[:,-2:]
    colunas_features_continuas = dados.iloc[:,13:-2].columns
    features_continuas = dados.groupby('PATIENT_VISIT_IDENTIFIER')[colunas_features_continuas].fillna(method='backfill').fillna(method='ffill')
    dados_limpos = pd.concat([features_categoricas, features_continuas, saida], axis=1)
    dados_limpos.columns = dados.columns
    return dados_limpos

In [21]:
dados_limpos = preenche_tabela(sirio_libanes)

Dentre as features dentro do dataframe `x` temos a coluna `AGE_PERCENTIL` que indica se a pessoa faz parte do conglomerado, por exemplo, de até 10 anos.

In [22]:
dados_limpos.AGE_PERCENTIL.unique()

array(['60th', '90th', '10th', '40th', '70th', '20th', '50th', '80th',
       '30th', 'Above 90th'], dtype=object)

Esse dado é quantitativo ordinal, isso porque uma pessoa de até 10 anos é mais nova do que uma pessoa até 40 anos.

O pandas tem um método chamado `astype()` que muda tipo da variável. Neste método pode-se passar um parâmetro `category` assim ele vai ver quais categorias absolutas (por exemplo o retorno do método `.unique()`) e o índice será o valor da categoria. Veja que o retorno do método `unique()` é um array com 10 elementos, então o método `astype('category')` irá retornar 9 categorias de 0 a 9, cada uma atribuida aos 10 elementos:

In [23]:
dados_limpos.AGE_PERCENTIL = dados_limpos.AGE_PERCENTIL.astype('category').cat.codes

Note que o primeiro elemento tem `60th` e a categoria dele é o $0$, 

In [24]:
dados_limpos

,PATIENT_VISIT_IDENTIFIER,AGE_ABOVE65,AGE_PERCENTIL,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,...,TEMPERATURE_DIFF,OXYGEN_SATURATION_DIFF,BLOODPRESSURE_DIASTOLIC_DIFF_REL,BLOODPRESSURE_SISTOLIC_DIFF_REL,HEART_RATE_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL,OXYGEN_SATURATION_DIFF_REL,WINDOW,ICU
0,0,1,5,0,0.0,0.0,0.0,0.0,1.0,1.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,0
1,0,1,5,0,0.0,0.0,0.0,0.0,1.0,1.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,2-4,0
2,0,1,5,0,0.0,0.0,0.0,0.0,1.0,1.0,...,-1.000000,-1.000000,-0.389967,0.407558,-0.230462,0.096774,-1.000000,-1.000000,4-6,0
3,0,1,5,0,0.0,0.0,0.0,0.0,1.0,1.0,...,-1.000000,-1.000000,-0.389967,0.407558,-0.230462,0.096774,-1.000000,-1.000000,6-12,0
4,0,1,5,0,0.0,0.0,0.0,0.0,1.0,1.0,...,-0.238095,-0.818182,-0.389967,0.407558,-0.230462,0.096774,-0.242282,-0.814433,ABOVE_12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1920,384,0,4,1,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,0
1921,384,0,4,1,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,2-4,0
1922,384,0,4,1,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,4-6,0
1923,384,0,4,1,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,6-12,0


Note acima que o resultado da feature `AGE_PERCENTIL` não é mais do tipo $90$th, agora é uma variável categórica.

In [25]:
dados_limpos.query('WINDOW == "0-2" and ICU == 1')

,PATIENT_VISIT_IDENTIFIER,AGE_ABOVE65,AGE_PERCENTIL,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,...,TEMPERATURE_DIFF,OXYGEN_SATURATION_DIFF,BLOODPRESSURE_DIASTOLIC_DIFF_REL,BLOODPRESSURE_SISTOLIC_DIFF_REL,HEART_RATE_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL,OXYGEN_SATURATION_DIFF_REL,WINDOW,ICU
5,1,1,8,1,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,1
85,17,0,3,0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,1
135,27,1,5,1,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,1
205,41,1,6,0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.666667,-0.858586,-0.689777,-0.399526,-0.891572,-1.000000,-0.663881,-0.858709,0-2,1
235,47,0,4,0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,1
240,48,0,5,0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,1
270,54,1,5,0,0.0,0.0,1.0,0.0,0.0,0.0,...,-1.000000,-1.000000,-0.890980,-0.841512,-1.000000,-1.000000,-1.000000,-1.000000,0-2,1
310,62,1,6,1,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,1
425,85,1,6,0,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.809524,-0.959596,-1.000000,-1.000000,-0.926209,-1.000000,-0.805768,-0.959631,0-2,1
450,90,0,4,0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,1


In [26]:
identificador_dos_pacientes_a_remover = dados_limpos.query('WINDOW == "0-2" and ICU == 1')['PATIENT_VISIT_IDENTIFIER'].values
dados_limpos = dados_limpos.query(f'PATIENT_VISIT_IDENTIFIER not in "{identificador_dos_pacientes_a_remover.tolist}"')

dados_limpos.describe()

,PATIENT_VISIT_IDENTIFIER,AGE_ABOVE65,AGE_PERCENTIL,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,...,RESPIRATORY_RATE_DIFF,TEMPERATURE_DIFF,OXYGEN_SATURATION_DIFF,BLOODPRESSURE_DIASTOLIC_DIFF_REL,BLOODPRESSURE_SISTOLIC_DIFF_REL,HEART_RATE_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL,OXYGEN_SATURATION_DIFF_REL,ICU
count,1925.000000,1925.000000,1925.000000,1925.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,...,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000
mean,192.000000,0.467532,4.319481,0.368831,0.108333,0.028125,0.097917,0.019792,0.128125,0.046875,...,-0.758319,-0.811824,-0.910237,-0.817464,-0.758728,-0.848568,-0.771306,-0.812740,-0.910107,0.267532
std,111.168431,0.499074,2.867348,0.482613,0.310882,0.165373,0.297279,0.139320,0.334316,0.211426,...,0.433037,0.296595,0.255396,0.301522,0.389168,0.246814,0.400915,0.295435,0.256035,0.442787
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000
25%,96.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000
50%,192.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000
75%,288.000000,1.000000,7.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.705882,-0.690476,-0.919192,-0.704088,-0.613423,-0.764790,-0.732494,-0.692177,-0.918403,1.000000
max,384.000000,1.000000,9.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [27]:
def prepare_window(rows):
    # se ao longo da coluna ['ICU'] tiver algum valor:
    if (np.any(rows['ICU'])):
        rows.loc[rows['WINDOW'] == '0-2', 'ICU'] = 1
    return rows.loc[rows['WINDOW'] == '0-2']

In [28]:
dados_limpos = dados_limpos.groupby('PATIENT_VISIT_IDENTIFIER').apply(prepare_window)

dados_limpos.head()

,,PATIENT_VISIT_IDENTIFIER,AGE_ABOVE65,AGE_PERCENTIL,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,...,TEMPERATURE_DIFF,OXYGEN_SATURATION_DIFF,BLOODPRESSURE_DIASTOLIC_DIFF_REL,BLOODPRESSURE_SISTOLIC_DIFF_REL,HEART_RATE_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL,OXYGEN_SATURATION_DIFF_REL,WINDOW,ICU
PATIENT_VISIT_IDENTIFIER,,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,5,0,0.0,0.0,0.0,0.0,1.0,1.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,1
1,5,1,1,8,1,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,1
2,10,2,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.000000,-0.959596,-0.515528,-0.351328,-0.747001,-0.756272,-1.000000,-0.961262,0-2,1
3,15,3,0,3,1,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,0
4,20,4,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.952381,-0.979798,-1.000000,-0.883669,-0.956805,-0.870968,-0.953536,-0.980333,0-2,0


# Separando dados Treino e Teste

In [29]:
colunas_quantitativas = dados_limpos.columns

sirio_libanes_quantitativos = dados_limpos[colunas_quantitativas].dropna()
sirio_libanes_quantitativos.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 384 entries, (0, 0) to (384, 1920)
Columns: 231 entries, PATIENT_VISIT_IDENTIFIER to ICU
dtypes: float64(225), int64(4), int8(1), object(1)
memory usage: 706.1+ KB


Varificar quais colunas ficaram de fora:

In [30]:
for coluna in sirio_libanes.columns:
    if coluna not in colunas_quantitativas:
        print(coluna)

Ou seja, estamos usando todas as features disponíveis. Agora vamos separar as variáveis independentes em `x` e as variáveis dependentes em `y`:

In [31]:
y = sirio_libanes_quantitativos['ICU']
x = sirio_libanes_quantitativos.drop(['ICU','WINDOW'], axis=1)

# Importando modelos

In [32]:
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC



# Importando métricas

In [33]:
from sklearn.metrics import accuracy_score

# Impotarndo metodo de estatificação

In [34]:
from sklearn.model_selection import train_test_split

In [35]:
%%time
seed = np.random.seed(9834520)

epocas = {}
for epoca in range(0,10):
    # Instanciando modelos
    modelo_dummy = DummyClassifier(strategy='prior')
    modelo_regressao_logistica = LogisticRegression(solver='newton-cg',max_iter=10000)
    modelo_svc = SVC(max_iter=5000, kernel='sigmoid')
    modelo_linear_svc = LinearSVC(max_iter=5000)
    
    # Separação dos dados
    x_treino, x_teste, y_treino, y_teste = train_test_split(x,y, random_state=seed)

    # Ajuste dos modelos
    modelo_dummy.fit(x_treino,y_treino)
    modelo_regressao_logistica.fit(x_treino,y_treino)
    modelo_svc.fit(x_treino, y_treino)
    modelo_linear_svc.fit(x_treino, y_treino)
    
    # Predição dos modelos
    y_pred_dummy = modelo_dummy.predict(x_teste)
    y_pred_logis = modelo_regressao_logistica.predict(x_teste)
    y_pred_svc = modelo_svc.predict(x_teste)
    y_pred_linear_svc = modelo_linear_svc.predict(x_teste)
    

  # Métricas de qualidade
    acc_dummy = accuracy_score(y_teste, y_pred_dummy)
    acc_logis = accuracy_score(y_teste, y_pred_logis)
    acc_svc = accuracy_score(y_teste, y_pred_svc)
    acc_linear_svc = accuracy_score(y_teste, y_pred_linear_svc)
    
    # Exibindo méétricas
    epocas[epoca] = (acc_dummy, acc_logis, acc_svc, acc_linear_svc)

for chave, valor in epocas.items():
    print(f'Epoca {chave} - '+'\033[34m'+f'Acurácia Dummy: {valor[0].round(2)*100:.2f}% | Acurácia Logistic: {valor[1].round(2)*100:.2f}% | Acurácia SVC: {valor[2].round(2) * 100:.2f} | Acurácia Linear SVC: {valor[3].round(2) * 100:.2f} '+'\033[0;0m')


epocas = pd.DataFrame(epocas, index=['Dummy','Logistic','SVC','LinearSVC']).round(3) * 100
descricao_epocas = epocas.T.describe().T

display(descricao_epocas)

/home/conrado/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/conrado/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/conrado/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/conrado/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/conrado/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the num

Epoca 0 - Acurácia Dummy: 47.00% | Acurácia Logistic: 76.00% | Acurácia SVC: 43.00 | Acurácia Linear SVC: 56.00 
Epoca 1 - Acurácia Dummy: 49.00% | Acurácia Logistic: 75.00% | Acurácia SVC: 57.00 | Acurácia Linear SVC: 60.00 
Epoca 2 - Acurácia Dummy: 49.00% | Acurácia Logistic: 69.00% | Acurácia SVC: 51.00 | Acurácia Linear SVC: 58.00 
Epoca 3 - Acurácia Dummy: 41.00% | Acurácia Logistic: 80.00% | Acurácia SVC: 57.00 | Acurácia Linear SVC: 73.00 
Epoca 4 - Acurácia Dummy: 41.00% | Acurácia Logistic: 72.00% | Acurácia SVC: 52.00 | Acurácia Linear SVC: 61.00 
Epoca 5 - Acurácia Dummy: 47.00% | Acurácia Logistic: 68.00% | Acurácia SVC: 45.00 | Acurácia Linear SVC: 57.00 
Epoca 6 - Acurácia Dummy: 50.00% | Acurácia Logistic: 68.00% | Acurácia SVC: 59.00 | Acurácia Linear SVC: 68.00 
Epoca 7 - Acurácia Dummy: 45.00% | Acurácia Logistic: 67.00% | Acurácia SVC: 50.00 | Acurácia Linear SVC: 60.00 
Epoca 8 - Acurácia Dummy: 50.00% | Acurácia Logistic: 66.00% | Acurácia SVC: 53.00 | Acurácia Li

/home/conrado/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


,count,mean,std,min,25%,50%,75%,max
Dummy,10.0,46.47,3.495728,40.6,45.325,46.90,49.00,50.0
Logistic,10.0,71.77,5.219632,65.6,67.700,70.35,75.75,80.2
SVC,10.0,52.71,5.823411,42.7,50.250,52.60,57.30,59.4
LinearSVC,10.0,61.86,5.696237,56.2,57.550,60.40,66.15,72.9


CPU times: user 12.4 s, sys: 714 ms, total: 13.1 s
Wall time: 10.1 s
